In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import arch ;
from arch import arch_model

In [2]:
%matplotlib inline
from pysabr import Hagan2002LognormalSABR
from pysabr import Hagan2002NormalSABR
from pysabr.helpers import year_frac_from_maturity_label
import os.path
import itertools

In [ ]:
# prepare swap rate and preprocess swaption dataframe

In [3]:
def convert_to_years(value):
    if value.endswith('y'):
        return int(value[:-1])
    else:
        months = int(value[:-1])
        years = months / 12
        return years
swaption["Expiry"]=swaption["Expiry"].apply(convert_to_years)
swaption["Tenor"]=swaption["Tenor"].apply(convert_to_years)
swaption

In [4]:
swaption = pd.read_csv("../data/market-data-swaption-vols.csv")

In [15]:
swaption['Date']=pd.to_datetime(swaption['Date'],format='%Y-%m-%d')
swap = pd.read_csv("../data/market-data-swap-rates.csv")
swap['Date']=pd.to_datetime(swap['Date'],format='%Y-%m-%d')
swap['Start Date']=pd.to_datetime(swap['Start Date'],format='%Y-%m-%d')

In [16]:
swap['f_e']=((swap['Start Date']-swap['Date'])/ np.timedelta64(1, 'Y')).round(1)

In [17]:
swap['Tenor']=swap['Tenor'].apply(convert_to_years)

In [27]:
swaption['Forward Swap Rate'] = None

# Loop through each row in swaption to find the matching Swap Rate from swap
for i in range(len(swaption)):
    # Define the matching conditions
    condition = (
        (swap['Date'] == swaption.loc[i, 'Date']) &
        (swap['Tenor'] == swaption.loc[i, 'Tenor']) &
        (swap['f_e'] == swaption.loc[i, 'Expiry'])
    )
    # Find the corresponding Swap Rate from swap
    matching_rate = swap.loc[condition, 'Swap Rate']
    
    # Assign the found Swap Rate to the Forward Swap Rate column in swaption
    if not matching_rate.empty:
        swaption.loc[i, 'Forward Swap Rate'] = matching_rate.values[0]

In [44]:
# Create a dictionary to map the string values to the corresponding multipliers
strike_map = {'atm': 1.0, 'atm-1.0%': 0.99, 'atm-0.5%': 0.995, 'atm+0.5%': 1.005, 'atm+1.0%': 1.01}

# Create a new column 'Numerical Strike' in the swaption DataFrame
swaption['Numerical Strike'] = np.where(swaption['Strike'].isin(strike_map.keys()),
                                        swaption['Forward Swap Rate'] * swaption['Strike'].map(strike_map),
                                        np.nan)

In [182]:
swaption.to_csv('swaption1.csv')

In [20]:
# swaption
swap

,Date,Start Date,Tenor,Swap Rate,f_e
0,2021-01-13,2022-01-13,10.0,1.081333,1.0
1,2021-01-13,2022-01-13,15.0,1.265960,1.0
2,2021-01-13,2022-01-13,1.5,0.221625,1.0
3,2021-01-13,2022-01-13,1.0,0.195474,1.0
4,2021-01-13,2022-01-13,2.0,0.279370,1.0
...,...,...,...,...,...
63175,2024-01-12,2039-01-12,2.0,3.424505,15.0
63176,2024-01-12,2039-01-12,3.0,3.393758,15.0
63177,2024-01-12,2039-01-12,4.0,3.358897,15.0
63178,2024-01-12,2039-01-12,5.0,3.319531,15.0


In [45]:
swaption

,Date,Expiry,Tenor,Strike,Vols,Forward Swap Rate,Numerical Strike
0,2021-01-13,10.0,10.0,atm-1.0%,0.170552,1.656352,1.639789
1,2021-01-13,10.0,10.0,atm-0.5%,0.135412,1.656352,1.648071
2,2021-01-13,10.0,10.0,atm,0.114587,1.656352,1.656352
3,2021-01-13,10.0,10.0,atm+0.5%,0.137742,1.656352,1.664634
4,2021-01-13,10.0,10.0,atm+1.0%,0.169685,1.656352,1.672916
...,...,...,...,...,...,...,...
315895,2024-01-12,8.0,8.0,atm-1.0%,0.259523,3.440227,3.405825
315896,2024-01-12,8.0,8.0,atm-0.5%,0.243703,3.440227,3.423026
315897,2024-01-12,8.0,8.0,atm,0.239538,3.440227,3.440227
315898,2024-01-12,8.0,8.0,atm+0.5%,0.262953,3.440227,3.457428
